In [1]:
import cv2
from ultralytics import YOLO, solutions
import supervision as sv
import numpy as np
import pandas as pd

In [2]:
video_path = r'C:\Users\Mohamed Hamde\Data Science\Computer Vision\vehicles-2.mp4'

In [3]:
model = YOLO('yolov8n.pt')

In [63]:
video_info = sv.VideoInfo.from_video_path(video_path)
video_info

VideoInfo(width=1920, height=1080, fps=29, total_frames=1275)

In [185]:
# track object
track = sv.ByteTrack()

# bounding box and label objects
bounding_box_annotator = sv.BoundingBoxAnnotator()
label_annoator = sv.LabelAnnotator()

# line zone
start_point = sv.Point(170, 800)
end_point = sv.Point(1700, 800)
line_zone = sv.LineZone(start_point, end_point)
line_zone_annotator = sv.LineZoneAnnotator(custom_out_text = 'Number of Cars In', 
                                           custom_in_text = 'Number of Cars Out', 
                                           display_in_count=False, display_out_count=False,
                                           text_padding = 10, text_scale=1, text_offset=2, color=sv.draw.color.Color(255,0,0))

# callbacks
def callback(frame: np.ndarray, index:int) -> np.ndarray:
    # detections
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = track.update_with_detections(detections = detections[(detections.class_id == 2) | (detections.class_id == 7)])
    line_zone.trigger(detections)
    
    # as data frame
    ds_counting = {'count_in':[line_zone.out_count], 'count_out':[line_zone.in_count]}
    global df_counting
    df_counting = pd.DataFrame(ds_counting)

    # speed estimation
    prev_center = None
    prev_time = None
    for box in detections:
        x1, y1, x2, y2 = box[0]
        track_id = box[4]
        center_x = (x1 + x2) / 2 
        center_y = (y1 + y2) / 2  
        center = (center_x, center_y) 
        
        # Calculate speed if we have a previous center and time
        if prev_center is not None:
            displacement = abs(center[0] - prev_center[0])  # Assuming tracking along x-axis
            time_interval = 1 / video_info.fps  # Time between frames
            speed_mps = displacement / time_interval  # Speed in pixels per second (assuming 1 pixel = 1 meter)
            speed_kph = speed_mps * 3.6 # Speed in km/h
            
            font = cv2.FONT_HERSHEY_DUPLEX 
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)
            cv2.putText(frame, f"Speed: {speed_kph:.2f} km/h", (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
        # Update previous center and time
        prev_center = center  
        
    # customization input and output cars 
    font = cv2.FONT_HERSHEY_DUPLEX 
    cv2.putText(frame, f'Number of Cars Out: {line_zone.in_count}', (500, 550),  font, 1,  (255, 255, 0),  3) 
    cv2.putText(frame, f'Number of Cars In: {line_zone.out_count}', (1000, 550),  font, 1,  (255, 255, 0),  3)
    cv2.putText(frame, 'Line Zone', (170, 780),  font, 1.25,  (255, 255, 0),  3)
    
    # annotatios
    labels = [f'{model.model.names[class_id]}' for class_id in detections.class_id]
    annotated_frame = bounding_box_annotator.annotate(scene=frame, detections=detections)
    annotated_frame = label_annoator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    annotated_frame = line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

    return annotated_frame

In [187]:
sv.process_video(
    source_path = video_path,
    target_path = 'sv_counting_speeding_cars.mp4',
    callback=callback
)